# Puzzle 11
## Part 1


--- Day 11: Dumbo Octopus ---
You enter a large cavern full of rare bioluminescent dumbo octopuses! They seem to not like the Christmas lights on your submarine, so you turn them off for now.


There are 100 octopuses arranged neatly in a 10 by 10 grid. Each octopus slowly gains **energy** over time and **flashes** brightly for a moment when its energy is full. Although your lights are off, maybe you could navigate through the cave without disturbing the octopuses if you could predict when the flashes of light will happen.


Each octopus has an **energy level** - your submarine can remotely measure the energy level of each octopus (your puzzle input). For example:
<pre>
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526</pre>

The energy level of each octopus is a value between 0 and 9. Here, the top-left octopus has an energy level of 5, the bottom-right one has an energy level of 6, and so on.


You can model the energy levels and flashes of light in **steps**. During a single step, the following occurs:


* First, the energy level of each octopus increases by 1.
* Then, any octopus with an energy level greater than 9 **flashes**. This increases the energy level of all adjacent octopuses by 1, including octopuses that are diagonally adjacent. If this causes an octopus to have an energy level greater than 9, it **also flashes**. This process continues as long as new octopuses keep having their energy level increased beyond 9. (An octopus can only flash **at most once per step**.)
* Finally, any octopus that flashed during this step has its energy level set to 0, as it used all of its energy to flash.


Adjacent flashes can cause an octopus to flash on a step even if it begins that step with very little energy. Consider the middle octopus with 1 energy in this situation:

Before any steps:

11111<br>
19991<br>
19191<br>
19991<br>
11111<br>

After step 1:

34543<br>
4**000**4<br>
5**000**5<br>
4**000**4<br>
34543<br>

After step 2:

45654<br>
51115<br>
61116<br>
51115<br>
45654<br>

An octopus is **highlighted** when it flashed during the given step.

Here is how the larger example above progresses:

<pre>
Before any steps:
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526

After step 1:
6594254334
3856965822
6375667284
7252447257
7468496589
5278635756
3287952832
7993992245
5957959665
6394862637

After step 2:
8807476555
5089087054
8597889608
8485769600
8700908800
6600088989
6800005943
0000007456
9000000876
8700006848</pre>

After 100 steps, there have been a total of **1656** flashes.

Given the starting energy levels of the dumbo octopuses in your cavern, simulate 100 steps. **How many total flashes are there after 100 steps?**

In [42]:
import numpy as np
import pandas as pd

#test data
pz11_test = np.array([5483143223, 
                      2745854711,
                      5264556173,
                      6141336146,
                      6357385478,
                      4167524645,
                      2176841721,
                      6882881134,
                      4846848554,
                      5283751526])

#split each line to separate digits
pz11_test = np.array([[int(i) for i in line] for line in pz11_test.astype('str')])
pz11_test

#puzzle input
pz11_data = pd.read_csv("data/pz11.txt", header=None).values.flatten()
pz11_data = np.array([[int(i) for i in line] for line in pz11_data.astype('str')])

In [119]:
#count n flashes during x steps
def n_flashes(data, x_steps):
    #pad the data
    data_pad = np.pad(data,((1,1),(1,1)), constant_values=-9999)
    
    #grid indexes
    grid_i = [i for i in range(1,11)]
    flashes = 0
    for step in range(x_steps):
        data_pad += np.ones((12,12), dtype='int64') #step: add 1 to all elements

        if 10 in data_pad:
            
            while 10 in data_pad:
                data_pad = np.where(data_pad==10, 0, data_pad) #convert 10-s to 0-s
                
                for row_i in grid_i:
                    for col_i in grid_i:
                        r1,r2 = row_i - 1, row_i + 2
                        c1,c2 = col_i - 1, col_i + 2
                        if data_pad[row_i,col_i] == 0:
                            old_3x3 = data_pad[r1:r2,c1:c2]

                            #add increment to main matrix
                            data_pad[r1:r2,c1:c2] += np.where(old_3x3 == 0, 0, 1)
                            
                            flashes += 1
                
                #mark already flashed with -999            
                data_pad = np.where(data_pad==0,-999,data_pad)
                #cap values at 10
                data_pad = np.where(data_pad >= 10, 10, data_pad)
                
            
            data_pad[1:11,1:11] = np.where(data_pad[1:11,1:11] < 0, 0, 
                                           data_pad[1:11,1:11])
        else:
            continue
    
    print(data_pad[1:11,1:11],"\n")
    print("With %d steps %d flashes." % (x_steps, flashes))
    
n_flashes(pz11_test, 100)

[[0 3 9 7 6 6 6 8 6 6]
 [0 7 4 9 7 6 6 9 1 8]
 [0 0 5 3 9 7 6 9 3 3]
 [0 0 0 4 2 9 7 8 2 2]
 [0 0 0 4 2 2 9 8 9 2]
 [0 0 5 3 2 2 2 8 7 7]
 [0 5 3 2 2 2 2 9 6 6]
 [9 3 2 2 2 2 8 9 6 6]
 [7 9 2 2 2 8 6 8 6 6]
 [6 7 8 9 9 9 8 7 6 6]] 

With 100 steps 1656 flashes.


In [120]:
n_flashes(pz11_data, 100)

[[4 5 6 8 6 5 5 5 6 3]
 [5 7 1 1 8 6 5 5 6 6]
 [7 2 1 1 1 8 6 5 5 5]
 [0 4 2 1 1 1 8 6 5 5]
 [0 0 4 2 1 1 1 7 5 5]
 [0 0 0 4 2 1 1 8 5 5]
 [0 0 0 0 4 2 1 8 5 5]
 [0 0 0 0 0 4 2 8 5 5]
 [0 0 0 0 0 0 4 8 5 5]
 [9 0 0 0 0 0 0 7 5 5]] 

With 100 steps 1594 flashes.


## Part II
--- Part Two ---
It seems like the individual flashes aren't bright enough to navigate. However, you might have a better option: the flashes seem to be **synchronizing**!


In the example above, the first time all octopuses flash simultaneously is step 195:

<pre>
After step 193:

5877777777
8877777777
7777777777
7777777777
7777777777
7777777777
7777777777
7777777777
7777777777
7777777777

After step 194:
6988888888
9988888888
8888888888
8888888888
8888888888
8888888888
8888888888
8888888888
8888888888
8888888888

After step 195:
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
</pre>


If you can calculate the exact moments when the octopuses will all flash simultaneously, you should be able to navigate through the cavern. **What is the first step during which all octopuses flash?**

In [123]:
n_flashes(pz11_test, 195)

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]] 

With 195 steps 3125 flashes.


In [126]:
def first_sync(data):
    #pad the data
    data_pad = np.pad(data,((1,1),(1,1)), constant_values=-9999)
    
    #grid indexes
    grid_i = [i for i in range(1,11)]
    n_flash = 0
    steps = 0
    while n_flash < 100:
        steps += 1
        data_pad += np.ones((12,12), dtype='int64') #step: add 1 to all elements

        if 10 in data_pad:
            
            while 10 in data_pad:
                data_pad = np.where(data_pad==10, 0, data_pad) #convert 10-s to 0-s
                
                for row_i in grid_i:
                    for col_i in grid_i:
                        r1,r2 = row_i - 1, row_i + 2
                        c1,c2 = col_i - 1, col_i + 2
                        if data_pad[row_i,col_i] == 0:
                            old_3x3 = data_pad[r1:r2,c1:c2]

                            #add increment to main matrix
                            data_pad[r1:r2,c1:c2] += np.where(old_3x3 == 0, 0, 1)
                
                #mark already flashed with -999            
                data_pad = np.where(data_pad==0,-999,data_pad)
                
                main_matrix = data_pad[1:11, 1:11]
                n_flash = np.sum(main_matrix < 0)
                
                if n_flash == 100:
                    break
                
                #cap values at 10
                data_pad = np.where(data_pad >= 10, 10, data_pad)
                
            
            data_pad[1:11,1:11] = np.where(data_pad[1:11,1:11] < 0, 0, 
                                           data_pad[1:11,1:11])
        else:
            continue
    
    print(data_pad[1:11,1:11],"\n")
    print("All flash at step %d" % steps)
    
first_sync(pz11_test)

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]] 

All flash at step 195


In [127]:
first_sync(pz11_data)

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]] 

All flash at step 437


# Puzzle 12
## Part I

--- Day 12: Passage Pathing ---
With your submarine's subterranean subsystems subsisting suboptimally, the only way you're getting out of this cave anytime soon is by finding a path yourself. Not just a path - the only way to know if you've found the **best** path is to find **all** of them.

Fortunately, the sensors are still mostly working, and so you build a rough map of the remaining caves (your puzzle input). For example:
<pre>
start-A
start-b
A-c
A-b
b-d
A-end
b-end</pre>

This is a list of how all of the caves are connected. You start in the cave named start, and your destination is the cave named end. An entry like b-d means that cave b is connected to cave d - that is, you can move between them.

So, the above cave system looks roughly like this:
<pre>
    start
    /   \
c--A-----b--d
    \   /
     end</pre>

Your goal is to find the number of distinct **paths** that start at start, end at end, and don't visit small caves more than once. There are two types of caves: **big** caves (written in uppercase, like A) and **small** caves (written in lowercase, like b). It would be a waste of time to visit any small cave more than once, but big caves are large enough that it might be worth visiting them multiple times. So, all paths you find should **visit small caves at most once**, and can **visit big caves any number of times**.

Given these rules, there are 10 paths through this example cave system:
<pre>
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,end
start,A,c,A,b,A,end
start,A,c,A,b,end
start,A,c,A,end
start,A,end
start,b,A,c,A,end
start,b,A,end
start,b,end</pre>

(Each line in the above list corresponds to a single path; the caves visited by that path are listed in the order they are visited and separated by commas.)

Note that in this cave system, cave d is never visited by any path: to do so, cave b would need to be visited twice (once on the way to cave d and a second time when returning from cave d), and since cave b is small, this is not allowed.

Here is a slightly larger example:
<pre>
dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc</pre>


The 19 paths through it are as follows:
<pre>
start,HN,dc,HN,end
start,HN,dc,HN,kj,HN,end
start,HN,dc,end
start,HN,dc,kj,HN,end
start,HN,end
start,HN,kj,HN,dc,HN,end
start,HN,kj,HN,dc,end
start,HN,kj,HN,end
start,HN,kj,dc,HN,end
start,HN,kj,dc,end
start,dc,HN,end
start,dc,HN,kj,HN,end
start,dc,end
start,dc,kj,HN,end
start,kj,HN,dc,HN,end
start,kj,HN,dc,end
start,kj,HN,end
start,kj,dc,HN,end
start,kj,dc,end</pre>

Finally, this even larger example has 226 paths through it:
<pre>
fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW</pre>


**How many paths through this cave system are there that visit small caves at most once?**

In [157]:
#test set
pz12_test = pd.read_csv("data/pz12_test.txt", header=None)
pz12_test = pz12_test[0].str.split("-", n=1, expand=True)
pz12_test

,0,1
0,dc,end
1,HN,start
2,start,kj
3,dc,start
4,dc,HN
5,LN,dc
6,HN,end
7,kj,sa
8,kj,HN
9,kj,dc


In [218]:
left = pz12_test[0].to_list()
right = pz12_test[1].to_list()

#find unique strings except "end"
uniques = list(set(left) | set(right))
uniques.remove('end')


links = {key:[] for key in uniques} #empty dictionary
for l,r in zip(left,right):
    if r == 'start':
        links['start'].append(l)
    elif l == 'start':
        links['start'].append(r)
    elif l == 'end':
        links[r].append('end')
    elif r == 'end':
        links[l].append('end')
    else:
        links[l].append(r)
        links[r].append(l)

#remove dead-end keys
links_keys_rem = {}
vals_to_remove = []


for k,v in links.items():
    if (len(v) == 1 & v[0].islower()) and v[0] != 'end':
        vals_to_remove.append(k)
    else:
        links_keys_rem[k] = v
        
#remove dead-end values
links_final = {}
for k,v in links_keys_rem.items():
    links_final[k] = list(set(v).difference(set(vals_to_remove)))

#one-ways
lowers = [k for k,v in links_final.items() if k != 'start' and k.islower()]
#upper
uppers = [k for k,v in links_final.items() if k.isupper()]

print(uppers)
links_final

['HN']


{'kj': ['HN', 'dc'],
 'dc': ['HN', 'kj', 'end'],
 'start': ['HN', 'kj', 'dc'],
 'HN': ['dc', 'kj', 'end']}

In [202]:
def n_paths(dictionary):
    
    for dest in dictionary['start']:
        


['HN', 'dc']

kj ['sa', 'HN', 'dc']
dc ['end', 'HN', 'LN', 'kj']
start ['HN', 'kj', 'dc']
HN ['dc', 'end', 'kj']
sa ['kj']
LN ['dc']


In [136]:
from itertools import permutations, combinations_with_replacement

In [ ]:
dc-end
LN-dc
HN-end
kj-sa
kj-dc